In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
from math import sin, cos, sqrt, atan2, radians

# 经纬度算距离
def dist(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def rnum(): # 随机数值
    return random.uniform(1.5, 2)

In [3]:
# 添加数值，满足标题要求 
def creat_df():
    data = pd.read_csv('./data20180918n.txt', sep=",")
    sta_lat = [] # 路段起始经度
    sta_lon = [] # 路段起始维度
    end_lat = [] # 路段终点经度
    end_lon = [] # 路段终点维度
    dic = {}
    for each in data.LEN.unique():
        dic[each] = [rnum(), rnum(), rnum(), rnum()]

    for each in data.LEN:
        sta_lat.append(dic[each][0])
        sta_lon.append(dic[each][1])
        end_lat.append(dic[each][2])
        end_lon.append(dic[each][3])

    data['START_LATITUDE'] = sta_lat
    data['START_LONGTITUDE'] = sta_lon
    data['END_LATITUDE'] = end_lat
    data['END_LONGTITUDE'] = end_lon
    
    speed = []
    for each in data.SPEED:
        try:
            val = round(float(each), 2)
        except:
            val = np.nan
        speed.append(val)
    data.SPEED = speed
    
    time = []
    for each in data.UPDATE_TIME:
        try:
            val = str(each)
        except:
            val = np.nan
        time.append(val)
    data.UPDATE_TIME = time

    return data

In [4]:
data = creat_df()
data.head()

,RTICIDNEW,LEN,FUNCLASS,UPDATE_TIME,SPEED,START_LATITUDE,START_LONGTITUDE,END_LATITUDE,END_LONGTITUDE
0,46607520302,327.36906,4,201809180001,53.51,1.802179,1.760919,1.684975,1.551876
1,46607520302,327.36906,4,201809180002,56.06,1.802179,1.760919,1.684975,1.551876
2,46607520302,327.36906,4,201809180004,56.06,1.802179,1.760919,1.684975,1.551876
3,46607520302,327.36906,4,201809180005,56.06,1.802179,1.760919,1.684975,1.551876
4,46607520302,327.36906,4,201809180007,49.05,1.802179,1.760919,1.684975,1.551876


In [5]:
# 时间维度问题：
    # 路段如果相差不大，而且路段登记一致，说明大概速度一样，用上面路段进行时间速度补全
    # 若路段相差很大或路段登记不一致，则用该种路段的每时刻平均速度作为补全对象

# 空间维度问题：
    # 假设每条路都是连通的，则根据实际，每条路的起始点(或终点)与相邻的路的起始点(或终点) 不会相隔超过1km
    # 如果超过1km，则需要补全该路径，其起始点设为其前一条路的终点，其终点设为离前一条路路段最近的起始点

In [6]:
# 处理空间维度
def get_lost_location():
    data = creat_df() # create data
    dic = {}
    for indx, row in data.iterrows():
        dic[row.LEN] = [row.START_LATITUDE, row.START_LONGTITUDE, row.END_LATITUDE, row.END_LONGTITUDE]
        
    all_location = [value for key, value in dic.items()] # 所有坐标
    
    j = 0
    total_list = list(data.LEN.unique()) # 所有len（代表地点）
    candi = 'None'
    candidate = 'None'
    dic_dist = {each:{'get':[],'near':[],'mylocation':[], 'dis': '', 'neighbor':''} for each in total_list}

    for i in range(len(all_location)-1):
        stop = False
        j = i
        while not stop or dic_dist[total_list[i]]['get'] == []: # 如果有公路相连，则停止，目的寻找独立公路
            j+=1
            if j > len(all_location)-1:
                if dic_dist[total_list[i]]['get'] == []:
                        dic_dist[total_list[i]]['near'] = candidate
                        dic_dist[total_list[i]]['mylocation'] = [loca[2], loca[3]]
                        dic_dist[total_list[i]]['dis'] = lon
                        dic_dist[total_list[i]]['neighbor'] = neighbor
                stop = True
                break
            loca = all_location[i]
            lon = dist(all_location[j][0], all_location[j][1], loca[2], loca[3]) # 首尾相连END-START 距离 此处默认原数据start/end遵守一定方向规则

            if lon < 1:
                dic_dist[total_list[i]]['get'].append(total_list[j]) # record if distance smaller than 1km
                dic_dist[total_list[j]]['get'].append(total_list[i])
            if candi =='None' or candi > lon: # 找到当前距离最小的点
                candi = lon
                candidate = [all_location[j][0], all_location[j][1]]
                neighbor = total_list[j]
    return data, dic_dist, total_list
    
    
    

In [7]:
# 创造未连通的路，处理空间不均匀
def generate_rd():
    data, dic_dist, total_list = get_lost_location()
    
    length = data.RTICIDNEW.unique()[-1] # 路段最后的编号
    rd_location = [(key, value['mylocation'], value['near'], value['neighbor'])for key, value in dic_dist.items() if value['get'] == []]
    rti_list = [i for i in range(length+1, length +len(rd_location)+1)]# 按顺序创造路况编号
    rd_LEN = [dist(each[1][0], each[1][1],each[2][0], each[2][1]) for each in rd_location] # 路程长短

    # 道路等级, 根据最近的那条路
    rd_indx = [total_list.index(each[3]) for each in rd_location]
    rd_level = [data.FUNCLASS[each] for each in rd_indx]

    up_time = [np.nan] * len(rd_LEN)
    speed_list = [np.nan] * len(rd_LEN)
    start_latist = []
    start_lode = []
    end_latist = []
    end_lode = []
    for each in rd_location:
        start_latist.append(each[1][0])
        start_lode.append(each[1][1])
        end_latist.append(each[2][0])
        end_lode.append(each[2][1])
    return pd.DataFrame({'RTICIDNEW': rti_list, 'LEN': rd_LEN, 'FUNCLASS': rd_level, 'SPEED':speed_list,
                     'START_LATITUDE':start_latist, 'START_LONGTITUDE': start_lode, 
                      'END_LATITUDE': end_latist, 'END_LONGTITUDE': end_lode}) #rd_df

In [8]:
rd_df = generate_rd()

In [9]:
#dic_dist = get_lost_location(data)
#d_df = generate_rd(data, dic_dist)
new_data = pd.concat([data, rd_df], ignore_index=True, sort=False)

In [10]:
df_na = new_data[new_data.isnull().any(axis=1)]
df_na = df_na.reset_index(drop=True)

In [ ]:
df_na.head()

In [ ]:
new_data = new_data.dropna()
new_data = new_data.reset_index(drop = True)
new_data.head()

In [13]:
final_data = new_data.copy()
final_data.head()

,RTICIDNEW,LEN,FUNCLASS,UPDATE_TIME,SPEED,START_LATITUDE,START_LONGTITUDE,END_LATITUDE,END_LONGTITUDE
0,46607520302,327.36906,4,201809180001,53.51,1.791744,1.540743,1.850334,1.748049
1,46607520302,327.36906,4,201809180002,56.06,1.791744,1.540743,1.850334,1.748049
2,46607520302,327.36906,4,201809180004,56.06,1.791744,1.540743,1.850334,1.748049
3,46607520302,327.36906,4,201809180005,56.06,1.791744,1.540743,1.850334,1.748049
4,46607520302,327.36906,4,201809180007,49.05,1.791744,1.540743,1.850334,1.748049


In [14]:
# 规范化 时间
def get_newdata(new_data):
    new_index = ['00:00-0:59', '01:00-01:59', '02:00-02:59', '03:00-03:59', '04:00-04:59', '05:00-05:59', '06:00-06:59',
                 '07:00-07:59', '08:00-08:59', '09:00-09:59', '10:00-10:59', '11:00-11:59', '12:00-12:59', '13:00-13:59',
                 '14:00-14:59', '15:00-15:59', '16:00-16:59', '17:00-17:59', '18:00-18:59', '19:00-19:59', '20:00-20:59',
                 '21:00-21:59', '22:00-22:59', '23:00-23:59']
    #new_data['Backup time'] = new_data['UPDATE_TIME']
    alist = list(new_data.UPDATE_TIME)
    for i in range(len(alist)):
        alist[i] = alist[i][-4:-2]
    new_data.UPDATE_TIME = alist

    for each in new_data.UPDATE_TIME.unique():
        indx = int(each)
        new_data.UPDATE_TIME = new_data.UPDATE_TIME.replace(each, new_index[indx])
    return new_data

In [22]:
new_data = get_newdata(new_data)


In [23]:
#new_data = get_newdata(new_data)
class_df = new_data[['FUNCLASS','UPDATE_TIME','SPEED']] # 同一道路条件，不同时间段的平均速度
class_df1 = class_df.groupby(['FUNCLASS','UPDATE_TIME']).mean()
class_df1

SPEED
FUNCLASS UPDATE_TIME           
2        00:00-0:59   36.005323
         01:00-01:59  35.987500
         02:00-02:59  37.345082
         03:00-03:59  34.232167
         04:00-04:59  34.499574
         05:00-05:59  37.256081
         06:00-06:59  34.608947
         07:00-07:59  30.704000
         08:00-08:59  28.297375
         09:00-09:59  32.518875
         10:00-10:59  27.286250
         11:00-11:59  30.816282
         12:00-12:59  28.955658
         13:00-13:59  30.346500
         14:00-14:59  29.028608
         15:00-15:59  28.538500
         16:00-16:59  27.524500
         17:00-17:59  23.057250
         18:00-18:59  27.632000
         19:00-19:59  28.092500
         20:00-20:59  29.598500
         21:00-21:59  30.882917
         22:00-22:59  35.196000
         23:00-23:59  33.548000
3        00:00-0:59   61.037730
         01:00-01:59  60.437533
         02:00-02:59  60.537823
         03:00-03:59  60.846114
         04:00-04:59  60.028751
         05:00-05:59  61.421559
...                         ...
6        18:00-18:59  27.237019
         19:00-19:59  27.188113
         20:00-20:59  28.056015
         21:00-21:59  28.033981
         22:00-22:59  31.130759
         23:00-23:59  31.751812
7        00:00-0:59   28.314301
         01:00-01:59  29.165025
         02:00-02:59  30.242914
         03:00-03:59  30.186975
         04:00-04:59  29.578119
         05:00-05:59  29.420097
         06:00-06:59  27.787004
         07:00-07:59  25.520431
         08:00-08:59  25.316642
         09:00-09:59  23.288686
         10:00-10:59  23.219085
         11:00-11:59  23.942428
         12:00-12:59  22.710711
         13:00-13:59  22.394539
         14:00-14:59  22.870098
         15:00-15:59  22.393769
         16:00-16:59  23.345313
         17:00-17:59  22.269540
         18:00-18:59  22.540230
         19:00-19:59  22.487146
         20:00-20:59  23.382111
         21:00-21:59  23.799686
         22:00-22:59  27.812093
         23:00-23:59  29.002128

[144 rows x 1 columns]

In [32]:
class_df1.query('FUNCLASS == 4')

SPEED
FUNCLASS UPDATE_TIME           
4        00:00-0:59   44.262569
         01:00-01:59  44.572425
         02:00-02:59  44.503041
         03:00-03:59  44.007113
         04:00-04:59  43.741391
         05:00-05:59  43.749741
         06:00-06:59  43.205189
         07:00-07:59  36.246852
         08:00-08:59  35.660758
         09:00-09:59  38.294544
         10:00-10:59  39.306271
         11:00-11:59  40.701728
         12:00-12:59  40.715026
         13:00-13:59  39.520636
         14:00-14:59  39.094020
         15:00-15:59  38.005630
         16:00-16:59  36.781786
         17:00-17:59  35.210623
         18:00-18:59  35.221480
         19:00-19:59  38.095446
         20:00-20:59  39.706154
         21:00-21:59  40.103728
         22:00-22:59  42.148245
         23:00-23:59  42.512985

In [24]:
def new_dataframe():
    return pd.DataFrame({'RTICIDNEW':[], 'LEN':[], 'FUNCLASS':[], 'UPDATE_TIME':[], 'SPEED':[], 
                         'START_LATITUDE':[], 'START_LONGTITUDE':[], 'END_LATITUDE':[], 'END_LONGTITUDE':[]})

def get_newf(df_na): # 新建立的dataframe 补全道路信息
    # 由于数据只有2018年9月18日 0-24点，以三分钟update_time为基础
    format1= "20180918"
    hour_list = [str(i) if i > 9 else "0" + str(i)for i in range(24)]
    time_list = [str(format1 + i + str(k*3 + 1)) if k > 2 else format1 + i + "0" + str(k*3 + 1) for i in hour_list for k in range(20)]

    # 建立一个新dataframe
    total_new_frame = new_dataframe()
    for indx, row in df_na.iterrows():
        spd_list =  list(class_df1.query('FUNCLASS=='+ str(row.FUNCLASS))['SPEED'])
        sp_list = [each for each in spd_list for i in range(20)]
        new_frame = pd.DataFrame({'RTICIDNEW':[row.RTICIDNEW] * 20*24, 'LEN':[row.LEN] * 20*24, 
                                  'FUNCLASS':[row.FUNCLASS] * 20*24, 'UPDATE_TIME': time_list, 'SPEED':sp_list, 
                             'START_LATITUDE':[row.START_LATITUDE]* 20*24, 'START_LONGTITUDE':[row.START_LONGTITUDE]* 20*24, 
                                  'END_LATITUDE':[row.END_LATITUDE]* 20*24, 'END_LONGTITUDE':[row.END_LONGTITUDE]* 20*24})
        total_new_frame = pd.concat([total_new_frame, new_frame], ignore_index=True)
    return total_new_frame

In [26]:
total_new_frame = get_newf(df_na)

In [27]:
total_new_frame.head()

,RTICIDNEW,LEN,FUNCLASS,UPDATE_TIME,SPEED,START_LATITUDE,START_LONGTITUDE,END_LATITUDE,END_LONGTITUDE
0,4.760056e+10,166.77773,6.0,201809180001,32.713863,1.518273,1.528398,1.826238,1.971072
1,4.760056e+10,166.77773,6.0,201809180004,32.713863,1.518273,1.528398,1.826238,1.971072
2,4.760056e+10,166.77773,6.0,201809180007,32.713863,1.518273,1.528398,1.826238,1.971072
3,4.760056e+10,166.77773,6.0,201809180010,32.713863,1.518273,1.528398,1.826238,1.971072
4,4.760056e+10,166.77773,6.0,201809180013,32.713863,1.518273,1.528398,1.826238,1.971072


In [33]:
def get_empy_df(new_data):
    new_index = ['00:00-0:59', '01:00-01:59', '02:00-02:59', '03:00-03:59', '04:00-04:59', '05:00-05:59', '06:00-06:59',
                 '07:00-07:59', '08:00-08:59', '09:00-09:59', '10:00-10:59', '11:00-11:59', '12:00-12:59', '13:00-13:59',
                 '14:00-14:59', '15:00-15:59', '16:00-16:59', '17:00-17:59', '18:00-18:59', '19:00-19:59', '20:00-20:59',
                 '21:00-21:59', '22:00-22:59', '23:00-23:59']

    uniq_rd = new_data.RTICIDNEW.unique()
    emp_df = pd.DataFrame({'RTICIDNEW':[], 'LEN':[], 'FUNCLASS':[], 'UPDATE_TIME':[], 'SPEED':[], 
                             'START_LATITUDE':[], 'START_LONGTITUDE':[], 'END_LATITUDE':[], 'END_LONGTITUDE':[]})
    for code in uniq_rd:
        dic_time = {each:0 for each in new_index}
        for each in new_data[new_data.RTICIDNEW == code]['UPDATE_TIME']:
            dic_time[each] += 1

        for key, value in dic_time.items():
            if value == 0:
                #print(code, key)
                leng = list(new_data[new_data.RTICIDNEW == code]['LEN'])[0]
                fun = list(new_data[new_data.RTICIDNEW == code]['FUNCLASS'])[0]

                # create time format
                format1= "20180918"
                time1 = key[:2]
                up_time = [str(format1 + time1 + str(k*3 + 1)) if k > 2 else format1 + time1 + "0" + str(k*3 + 1) for k in range(20)]


                sp_num = list(class_df1.query('FUNCLASS == ' + str(fun))['SPEED'])[int(time1)]

                st_lat = list(new_data[new_data.RTICIDNEW == code]['START_LATITUDE'])[0]
                st_lon = list(new_data[new_data.RTICIDNEW == code]['START_LONGTITUDE'])[0]

                ed_lat = list(new_data[new_data.RTICIDNEW == code]['END_LATITUDE'])[0]
                ed_lon = list(new_data[new_data.RTICIDNEW == code]['END_LONGTITUDE'])[0]
                c1_df = pd.DataFrame({'RTICIDNEW':[code]* 20, 'LEN':[leng]* 20, 'FUNCLASS':[fun]* 20, 'UPDATE_TIME': up_time, 
                              'SPEED':[sp_num]* 20, 'START_LATITUDE':[st_lat]* 20, 'START_LONGTITUDE':[st_lon]* 20, 
                              'END_LATITUDE':[ed_lat]* 20, 'END_LONGTITUDE':[ed_lon]* 20})

                emp_df = pd.concat([emp_df, c1_df], ignore_index=True)
    return emp_df
    

In [38]:
df = pd.concat([final_data, emp_df], ignore_index= True)
df = pd.concat([df, total_new_frame], ignore_index= True)

In [39]:
# 按照路段编号排序
df = df.sort_values(by=['RTICIDNEW', 'UPDATE_TIME'])
df = df.reset_index(drop= True)

df.RTICIDNEW = [str(int(each)) for each in df.RTICIDNEW] #change to string

,RTICIDNEW,LEN,FUNCLASS,UPDATE_TIME,SPEED,START_LATITUDE,START_LONGTITUDE,END_LATITUDE,END_LONGTITUDE


In [ ]:
df[df.duplicated()]